In order to run the following noteboooks, if you haven't done yet, you need to set the openai key inside .env file as `OPENAI_API_KEY`

In [8]:
import os
from openai import OpenAI
import numpy as np
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

API_KEY = os.getenv("DASHSCOPE_API_KEY")
assert API_KEY, "ERROR: API Key is missing"
client = OpenAI(
    api_key=API_KEY,
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)
model = "text-embedding-v3"

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Next, we are going to load the Embedding Index into a Pandas Dataframe. The Embedding Index is stored in a JSON file called `embedding_index_3m.json`. The Embedding Index contains the Embeddings for each of the YouTube transcripts up until late Oct 2023.

In [44]:
def load_dataset(source: str, num_rows: int) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    pdv = pd_vectors.drop(columns=["text"], errors="ignore").fillna("")
    return pdv[:num_rows] if num_rows > 0 else pdv

In [47]:
pdv = load_dataset(DATASET_NAME, 10)
len(pdv),pdv.head()

(10,
                                  speaker  \
 0  Seth Juarez, Josh Lovejoy, Sarah Bird   
 1  Seth Juarez, Josh Lovejoy, Sarah Bird   
 2  Seth Juarez, Josh Lovejoy, Sarah Bird   
 3  Seth Juarez, Josh Lovejoy, Sarah Bird   
 4  Seth Juarez, Josh Lovejoy, Sarah Bird   
 
                                                title      videoId     start  \
 0  You're Not Solving the Problem You Think You'r...  -tJQm4mSh1s  00:00:00   
 1  You're Not Solving the Problem You Think You'r...  -tJQm4mSh1s  00:03:07   
 2  You're Not Solving the Problem You Think You'r...  -tJQm4mSh1s  00:06:13   
 3  You're Not Solving the Problem You Think You'r...  -tJQm4mSh1s  00:09:21   
 4  You're Not Solving the Problem You Think You'r...  -tJQm4mSh1s  00:12:24   
 
    seconds                                            summary  \
 0        0  Join Seth Juarez as he discusses ethical conce...   
 1      187  In this video, the speaker discusses the chall...   
 2      373  The video discusses the limita

Next, we are going to create a function called `get_videos` that will search the Embedding Index for the query. The function will return the top 5 videos that are most similar to the query. The function works as follows:

1. First, a copy of the Embedding Index is created.
2. Next, the Embedding for the query is calculated using the OpenAI Embedding API.
3. Then a new column is created in the Embedding Index called `similarity`. The `similarity` column contains the cosine similarity between the query Embedding and the Embedding for each video segment.
4. Next, the Embedding Index is filtered by the `similarity` column. The Embedding Index is filtered to only include videos that have a cosine similarity greater than or equal to 0.75.
5. Finally, the Embedding Index is sorted by the `similarity` column and the top 5 videos are returned.

In [ ]:
# original
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int,
    embeddings_column: str = "ada_v2",
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors[embeddings_column].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int,
    embeddings: list
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = embeddings.apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

In [49]:
pd_vectors = load_dataset(DATASET_NAME,10)
pd_vectors.head(5)

,speaker,title,videoId,start,seconds,summary,ada_v2
0,"Seth Juarez, Josh Lovejoy, Sarah Bird",You're Not Solving the Problem You Think You'r...,-tJQm4mSh1s,00:00:00,0,Join Seth Juarez as he discusses ethical conce...,"[0.004357332363724, -0.028409153223037, 0.0111..."
1,"Seth Juarez, Josh Lovejoy, Sarah Bird",You're Not Solving the Problem You Think You'r...,-tJQm4mSh1s,00:03:07,187,"In this video, the speaker discusses the chall...","[-0.0038613036740570003, -0.004626247566193000..."
2,"Seth Juarez, Josh Lovejoy, Sarah Bird",You're Not Solving the Problem You Think You'r...,-tJQm4mSh1s,00:06:13,373,The video discusses the limitations of general...,"[0.00287682027556, -0.012365541420876001, 0.02..."
3,"Seth Juarez, Josh Lovejoy, Sarah Bird",You're Not Solving the Problem You Think You'r...,-tJQm4mSh1s,00:09:21,561,The video discusses the importance of consider...,"[0.015913352370262, 0.000721095071639, 0.02349..."
4,"Seth Juarez, Josh Lovejoy, Sarah Bird",You're Not Solving the Problem You Think You'r...,-tJQm4mSh1s,00:12:24,744,The video discusses the importance of understa...,"[5.447878720588051e-06, -0.011837740428745, 0...."


In [ ]:
# gemini
import time
from IPython.display import display
import pandas as pd

# This cell assumes:
# 1. 'pd_vectors' DataFrame is loaded (e.g., from cell 'fbe19342' or '4cb9b31a').
# 2. 'client' (OpenAI client for DashScope) is initialized (from cell 'b8348a13').
# 3. 'model' variable (e.g., "text-embedding-v3") is defined (from cell 'b8348a13').

def create_qwen_embeddings_for_column(
    dataframe: pd.DataFrame,
    num_texts: int = -1,
    text_column: str = 'summary',
    new_embedding_column: str = 'qwen',
    model_name: str = model, # Uses the global 'model' variable by default
    batch_size: int = 5  # Number of texts to process in each API call
) -> pd.DataFrame:
    """
    Creates embeddings for a specified text column in the DataFrame using the Qwen model.

    Args:
        dataframe: Pandas DataFrame containing the text column.
        num_texts: num of texts to process. If -1, processes all texts.
        text_column: Name of the column with text to embed.
        new_embedding_column: Name for the new column to store embeddings.
        model_name: Name of the embedding model to use (from DashScope).
        batch_size: Size of batches for API calls.

    Returns:
        A new Pandas DataFrame with an added column for embeddings.
    """
    result_df = dataframe.copy()
    # Initialize the new column with None to store list objects (embeddings)
    result_df[new_embedding_column] = pd.Series([None] * len(result_df), dtype=object)

    if num_texts < 0:
        num_texts = len(result_df)
    print(f"Starting embedding generation for {num_texts} texts in column '{text_column}' using model '{model_name}'.")

    for i in range(0, num_texts, batch_size):
        batch_slice = result_df.iloc[i:i + batch_size]
        
        texts_to_embed = []
        # Keep track of original DataFrame indices for texts in the current batch that are valid
        valid_original_indices_in_batch = [] 

        for original_idx, text_content in zip(batch_slice.index, batch_slice[text_column]):
            if isinstance(text_content, str) and text_content.strip():
                texts_to_embed.append(text_content)
                valid_original_indices_in_batch.append(original_idx)
        
        if not texts_to_embed:
            # print(f"Batch starting at index {i}: No valid texts to embed.")
            continue  # Skip if no valid text in this batch
            
        try:
            response = client.embeddings.create(
                input=texts_to_embed,
                model=model_name
            )
            # Assign embeddings back to the correct rows in result_df using original indices
            for embedding_idx, original_df_idx in enumerate(valid_original_indices_in_batch):
                embedding_value = response.data[embedding_idx].embedding
                # Ensure the embedding is a list to handle potential NumPy array from client
                # and to ensure correct assignment if original_df_idx is duplicated.
                if hasattr(embedding_value, 'tolist'): # Check if it's like a NumPy array
                    result_df.loc[original_df_idx, new_embedding_column] = embedding_value.tolist()
                else: # Assume it's already a list or compatible
                    result_df.iloc[original_df_idx, new_embedding_column] = embedding_value
            
            print(f"Processed batch covering original indices {i} to {min(i + batch_size - 1, num_texts - 1)}. "
                  f"{len(texts_to_embed)} embeddings generated in this batch.")            
        except Exception as e:
            print(f"Error processing batch covering original indices {i} to {min(i + batch_size - 1, num_texts - 1)}: {e}")
            # Embeddings for this batch will remain None for the affected rows if an error occurred
        
        if i + batch_size < num_texts: # Small delay between batches if not the last one
             time.sleep(0.5) # API politeness: adjust as needed

    print(f"Embedding generation finished for column '{text_column}'.")
    return result_df

# Check if 'pd_vectors' DataFrame exists in the environment
if 'pd_vectors' in locals() or 'pd_vectors' in globals():
    # Generate embeddings for the 'summary' column
    # The result is a new DataFrame. You can choose to overwrite pd_vectors if desired.
    print(f"Processing the first 10 summaries from 'pd_vectors'.")
    pd_vectors_with_summary_embeddings = create_qwen_embeddings_for_column(
        pd_vectors,
        num_texts=10,  # Process only the first 10 summaries for demonstration
        text_column='summary',
        new_embedding_column='summary_embedding_qwen', # Name of the new column
        model_name=model # Uses the global 'model' variable ("text-embedding-v3")
    )

    # Display information about the generated embeddings
    if not pd_vectors_with_summary_embeddings.empty and \
       'summary_embedding_qwen' in pd_vectors_with_summary_embeddings.columns:
        
        num_successfully_embedded = pd_vectors_with_summary_embeddings['summary_embedding_qwen'].notna().sum()
        total_texts = len(pd_vectors_with_summary_embeddings)
        print(f"\nSuccessfully generated embeddings for {num_successfully_embedded} out of {total_texts} summaries.")

        # Check the length of the first valid embedding
        first_valid_embedding_series = pd_vectors_with_summary_embeddings[
            pd_vectors_with_summary_embeddings['summary_embedding_qwen'].notna()
        ]['summary_embedding_qwen']
        
        if not first_valid_embedding_series.empty:
            first_embedding_vector = first_valid_embedding_series.iloc[0]
            print(f"Length of the first valid summary embedding vector: {len(first_embedding_vector)}")
        
        print("\nDataFrame with new Qwen summary embeddings (first 5 rows):")
        # Display relevant columns: 'summary' and the new 'summary_embedding_qwen'
        display(pd_vectors_with_summary_embeddings[['summary', 'summary_embedding_qwen']].head())
        
        # To update the original pd_vectors DataFrame, you can uncomment the following line:
        # pd_vectors = pd_vectors_with_summary_embeddings
    else:
        print("\nFailed to generate embeddings, or the new embedding column is missing from the result.")
else:
    print("\nError: 'pd_vectors' DataFrame not found. Please ensure it is loaded in a previous cell.")


Processing the first 10 summaries from 'pd_vectors'.
Starting embedding generation for 10 texts in column 'summary' using model 'text-embedding-v3'.
Error processing batch covering original indices 0 to 4: Must have equal len keys and value when setting with an iterable
Error processing batch covering original indices 5 to 9: Must have equal len keys and value when setting with an iterable
Embedding generation finished for column 'summary'.

Successfully generated embeddings for 0 out of 1409 summaries.

DataFrame with new Qwen summary embeddings (first 5 rows):


,summary,summary_embedding_qwen
0,Join Seth Juarez as he discusses ethical conce...,None
1,"In this video, the speaker discusses the chall...",None
2,The video discusses the limitations of general...,None
3,The video discusses the importance of consider...,None
4,The video discusses the importance of understa...,None


In [ ]:
# claude 
def create_embeddings_for_summaries(dataframe, model_name='text-embedding-v3'):
    """
    Create embeddings for all summary texts in the dataframe using the specified model.
    
    Args:
        dataframe: Pandas DataFrame containing a 'summary' column
        model_name: Name of the embedding model to use
        
    Returns:
        DataFrame with added 'summary_embeddings' column containing the embeddings
    """
    result_df = dataframe.copy()
    result_df['summary_embeddings'] = None
    result_embeddings = []
    
    print(f"Creating embeddings for {len(result_df)} summary texts...")
    
    # Process in batches to avoid rate limits and improve performance
    batch_size = 10
    for i in range(0, len(result_df), batch_size):
        batch = result_df.iloc[i:i+batch_size]
        summaries = batch['summary'].tolist()
        
        # Skip empty summaries
        valid_indices = []
        valid_summaries = []
        for idx, summary in enumerate(summaries):
            if isinstance(summary, str) and summary.strip():
                valid_indices.append(idx)
                valid_summaries.append(summary)
        
        if not valid_summaries:
            continue
            
        # Get embeddings for valid summaries
        response = client.embeddings.create(
            input=valid_summaries,
            model=model_name
        )
        
        # Update the dataframe with embeddings
        for local_idx, global_idx in enumerate(valid_indices):
            # result_df.iloc[i + global_idx, result_df.columns.get_loc('summary_embeddings')] = response.data[local_idx].embedding
            result_embeddings.append(response.data[local_idx].embedding)
        print(f"Processed {i + len(batch)} out of {len(result_df)} items")
    return result_embeddings
    # return result_df

# Create embeddings for all summaries
# pd_vectors_subset = pd_vectors.head(10).copy()  # For demonstration, using a subset
# pd_vectors_with_qwen_embeddings = create_embeddings_for_summaries(pd_vectors_subset)
# result_embeddings = create_embeddings_for_summaries(pd_vectors_subset)
# Display the first few rows to confirm embeddings were created
# print(f"First row summary embedding length: {len(pd_vectors_with_qwen_embeddings.iloc[0]['summary_embeddings'])}")
# pd_vectors_with_qwen_embeddings.head(2)

In [ ]:
pd_vectors = load_dataset(DATASET_NAME, 10)
result_embeddings = create_embeddings_for_summaries(pd_vectors)

In [52]:
len(result_embeddings),type(result_embeddings[0]), len(result_embeddings[0]),result_embeddings[0]

(10,
 list,
 1024,
 [-0.06689998507499695,
  -0.05340397730469704,
  0.015613218769431114,
  -0.026972681283950806,
  -0.09110772609710693,
  -0.02726271003484726,
  -0.03362400829792023,
  0.021829502657055855,
  -0.009121406823396683,
  0.026643982157111168,
  0.0004670672642532736,
  -0.013022294268012047,
  0.006332295946776867,
  0.008236818946897984,
  0.040294673293828964,
  -0.025580542162060738,
  0.038593169301748276,
  0.0008126849425025284,
  -0.05464143306016922,
  -0.06790541857481003,
  -0.044123053550720215,
  -0.002137754112482071,
  -0.036156926304101944,
  -0.010789072141051292,
  0.09667627513408661,
  0.03886386379599571,
  -0.04860883206129074,
  -0.05796709656715393,
  0.017353391274809837,
  -0.03841915354132652,
  -0.04420039430260658,
  0.04157079756259918,
  -0.0028882038313895464,
  -0.025889907032251358,
  -0.0654691755771637,
  -0.03053036890923977,
  -0.013650690205395222,
  -0.019045226275920868,
  -0.07560084760189056,
  -0.03122643753886223,
  0.012770

This function is very simple, it just prints out the results of the search query.

In [55]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. First, the Embedding Index is loaded into a Pandas Dataframe.
2. Next, the user is prompted to enter a query.
3. Then the `get_videos` function is called to search the Embedding Index for the query.
4. Finally, the `display_results` function is called to display the results to the user.
5. The user is then prompted to enter another query. This process continues until the user enters `exit`.

![](../images/notebook-search.png?WT.mc_id=academic-105485-koreyst)

You will be prompted to enter a query. Enter a query and press enter. The application will return a list of videos that are relevant to the query. The application will also return a link to the place in the video where the answer to the question is located.

Here are some queries to try out:

- What is Azure Machine Learning?
- How do convolutional neural networks work?
- What is a neural network?
- Can I use Jupyter Notebooks with Azure Machine Learning?
- What is ONNX?

In [57]:
# pd_vectors = load_dataset(DATASET_NAME,10)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5, result_embeddings)
    display_results(videos, query)

KeyError: "None of [Index([(-0.06689998507499695, -0.05340397730469704, 0.015613218769431114, -0.026972681283950806, -0.09110772609710693, -0.02726271003484726, -0.03362400829792023, 0.021829502657055855, -0.009121406823396683, 0.026643982157111168, 0.0004670672642532736, -0.013022294268012047, 0.006332295946776867, 0.008236818946897984, 0.040294673293828964, -0.025580542162060738, 0.038593169301748276, 0.0008126849425025284, -0.05464143306016922, -0.06790541857481003, -0.044123053550720215, -0.002137754112482071, -0.036156926304101944, -0.010789072141051292, 0.09667627513408661, 0.03886386379599571, -0.04860883206129074, -0.05796709656715393, 0.017353391274809837, -0.03841915354132652, -0.04420039430260658, 0.04157079756259918, -0.0028882038313895464, -0.025889907032251358, -0.0654691755771637, -0.03053036890923977, -0.013650690205395222, -0.019045226275920868, -0.07560084760189056, -0.03122643753886223, 0.012770935893058777, -0.00293654203414917, -0.01947060227394104, -0.085075125098

In [10]:
pd_vectors = load_dataset(DATASET_NAME)
pd_vectors.head(5)

,speaker,title,videoId,start,seconds,summary,ada_v2
0,"Seth Juarez, Josh Lovejoy, Sarah Bird",You're Not Solving the Problem You Think You'r...,-tJQm4mSh1s,00:00:00,0,Join Seth Juarez as he discusses ethical conce...,"[0.004357332363724, -0.028409153223037, 0.0111..."
1,"Seth Juarez, Josh Lovejoy, Sarah Bird",You're Not Solving the Problem You Think You'r...,-tJQm4mSh1s,00:03:07,187,"In this video, the speaker discusses the chall...","[-0.0038613036740570003, -0.004626247566193000..."
2,"Seth Juarez, Josh Lovejoy, Sarah Bird",You're Not Solving the Problem You Think You'r...,-tJQm4mSh1s,00:06:13,373,The video discusses the limitations of general...,"[0.00287682027556, -0.012365541420876001, 0.02..."
3,"Seth Juarez, Josh Lovejoy, Sarah Bird",You're Not Solving the Problem You Think You'r...,-tJQm4mSh1s,00:09:21,561,The video discusses the importance of consider...,"[0.015913352370262, 0.000721095071639, 0.02349..."
4,"Seth Juarez, Josh Lovejoy, Sarah Bird",You're Not Solving the Problem You Think You'r...,-tJQm4mSh1s,00:12:24,744,The video discusses the importance of understa...,"[5.447878720588051e-06, -0.011837740428745, 0...."
